<a href="https://colab.research.google.com/github/Hero0963/ML_2021/blob/main/ML_2021_hw02_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Download Data
!gdown --id '1HPkcmQmFGu-3OknddKIa5dNDsR05lIQR' --output data.zip
!unzip data.zip
!ls 

data.zip			       model.ckpt
hero_hw_02_2022_09_12-04_26_56_PM.csv  sample_data
hero_hw_02_2022_09_12-05_11_16_PM.csv  timit_11


In [ ]:
# Preparing Data
import numpy as np

print('Loading data ...')

data_root='./timit_11/'
train = np.load(data_root + 'train_11.npy')
train_label = np.load(data_root + 'train_label_11.npy')
test = np.load(data_root + 'test_11.npy')

print('Size of training data: {}'.format(train.shape))
print('Size of testing data: {}'.format(test.shape))

Loading data ...
Size of training data: (1229932, 429)
Size of testing data: (451552, 429)


In [ ]:
# Create Dataset
import torch
from torch.utils.data import Dataset

class TIMITDataset(Dataset):
    def __init__(self, X, y=None):
        self.data = torch.from_numpy(X).float()
        if y is not None:
            y = y.astype(np.int)
            self.label = torch.LongTensor(y)
        else:
            self.label = None

    def __getitem__(self, idx):
        if self.label is not None:
            return self.data[idx], self.label[idx]
        else:
            return self.data[idx]

    def __len__(self):
        return len(self.data)

In [ ]:
# Setup Validation Set
VAL_RATIO = 0.1

percent = int(train.shape[0] * (1 - VAL_RATIO))
train_x, train_y, val_x, val_y = train[:percent], train_label[:percent], train[percent:], train_label[percent:]
print('Size of training set: {}'.format(train_x.shape))
print('Size of validation set: {}'.format(val_x.shape))

Size of training set: (1106938, 429)
Size of validation set: (122994, 429)


In [ ]:
# Setup Batch Size
BATCH_SIZE = 64

from torch.utils.data import DataLoader

train_set = TIMITDataset(train_x, train_y)
val_set = TIMITDataset(val_x, val_y)
train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True) #only shuffle the training data
val_loader = DataLoader(val_set, batch_size=BATCH_SIZE, shuffle=False)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  if __name__ == '__main__':


In [ ]:
# GC ,since the data size is huge
import gc

del train, train_label, train_x, train_y, val_x, val_y
gc.collect()

44

In [ ]:
# Create Model
import torch
import torch.nn as nn

class Classifier(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()
        self.layer_0 = nn.Linear(429, 2048)
        self.layer_1 = nn.Linear(2048, 1024)
        self.layer_2 = nn.Linear(1024, 512)
        self.layer_3 = nn.Linear(512, 128)
        self.out = nn.Linear(128, 39) 

        self.act_fn = nn.ReLU()
        self.dropout = nn.Dropout(0.25)

        self.batchnorm_0 = nn.BatchNorm1d(2048)
        self.batchnorm_1 = nn.BatchNorm1d(1024)
        self.batchnorm_2 = nn.BatchNorm1d(512)
        self.batchnorm_3 = nn.BatchNorm1d(128)

    def forward(self, x):
        x = self.layer_0(x)
        x = self.batchnorm_0(x)
        x = self.act_fn(x)
        x = self.dropout(x)


        x = self.layer_1(x)
        x = self.batchnorm_1(x)
        x = self.act_fn(x)
        x = self.dropout(x)

        x = self.layer_2(x)
        x = self.batchnorm_2(x)
        x = self.act_fn(x)
        x = self.dropout(x)

        x = self.layer_3(x)
        x = self.batchnorm_3(x)
        x = self.act_fn(x)
        x = self.dropout(x)

        x = self.out(x)
        
        return x

In [ ]:
# Training
#check device
def get_device():
  return 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
# fix random seed
def same_seeds(seed):
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)  
    np.random.seed(seed)  
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

In [ ]:
# fix random seed for reproducibility
my_lucky_number=10963
same_seeds(my_lucky_number)

# get device 
device = get_device()
print(f'DEVICE: {device}')

# training parameters
num_epoch = 20               # number of training epoch
learning_rate = 0.0001       # learning rate

# the path where checkpoint saved
model_path = './model.ckpt'

# create model, define a loss function, and optimizer
model = Classifier().to(device)
criterion = nn.CrossEntropyLoss() 
optimizer = torch.optim.Adam(model.parameters())  # default

DEVICE: cuda


In [ ]:
# start training

best_acc = 0.0
for epoch in range(num_epoch):
    train_acc = 0.0
    train_loss = 0.0
    val_acc = 0.0
    val_loss = 0.0

    # training
    model.train() # set the model to training mode
    for i, data in enumerate(train_loader):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad() 
        outputs = model(inputs)  
        batch_loss = criterion(outputs, labels)
        _, train_pred = torch.max(outputs, 1) # get the index of the class with the highest probability
        batch_loss.backward() 
        optimizer.step() 

        train_acc += (train_pred.cpu() == labels.cpu()).sum().item()
        train_loss += batch_loss.item()

    # validation
    if len(val_set) > 0:
        model.eval() # set the model to evaluation mode
        with torch.no_grad():
            for i, data in enumerate(val_loader):
                inputs, labels = data
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                batch_loss = criterion(outputs, labels) 
                _, val_pred = torch.max(outputs, 1) 
            
                val_acc += (val_pred.cpu() == labels.cpu()).sum().item() # get the index of the class with the highest probability
                val_loss += batch_loss.item()

            print('[{:03d}/{:03d}] Train Acc: {:3.6f} Loss: {:3.6f} | Val Acc: {:3.6f} loss: {:3.6f}'.format(
                epoch + 1, num_epoch, train_acc/len(train_set), train_loss/len(train_loader), val_acc/len(val_set), val_loss/len(val_loader)
            ))

            # if the model improves, save a checkpoint at this epoch
            if val_acc > best_acc:
                best_acc = val_acc
                torch.save(model.state_dict(), model_path)
                print('saving model with acc {:.3f}'.format(best_acc/len(val_set)))
    else:
        print('[{:03d}/{:03d}] Train Acc: {:3.6f} Loss: {:3.6f}'.format(
            epoch + 1, num_epoch, train_acc/len(train_set), train_loss/len(train_loader)
        ))

# if not validating, save the last epoch
if len(val_set) == 0:
    torch.save(model.state_dict(), model_path)
    print('saving model at last epoch')

[001/020] Train Acc: 0.605809 Loss: 1.279852 | Val Acc: 0.695676 loss: 0.943210
saving model with acc 0.696
[002/020] Train Acc: 0.667072 Loss: 1.052570 | Val Acc: 0.716401 loss: 0.864113
saving model with acc 0.716
[003/020] Train Acc: 0.689360 Loss: 0.971918 | Val Acc: 0.727515 loss: 0.826697
saving model with acc 0.728
[004/020] Train Acc: 0.703292 Loss: 0.920539 | Val Acc: 0.733629 loss: 0.804935
saving model with acc 0.734
[005/020] Train Acc: 0.713446 Loss: 0.883456 | Val Acc: 0.738556 loss: 0.789812
saving model with acc 0.739
[006/020] Train Acc: 0.722130 Loss: 0.853380 | Val Acc: 0.743410 loss: 0.774215
saving model with acc 0.743
[007/020] Train Acc: 0.728952 Loss: 0.827242 | Val Acc: 0.744492 loss: 0.771584
saving model with acc 0.744
[008/020] Train Acc: 0.734779 Loss: 0.806519 | Val Acc: 0.743679 loss: 0.775195
[009/020] Train Acc: 0.739503 Loss: 0.789370 | Val Acc: 0.747053 loss: 0.765161
saving model with acc 0.747
[010/020] Train Acc: 0.744178 Loss: 0.773349 | Val Acc: 

In [ ]:
# Testing
# create testing dataset
test_set = TIMITDataset(test, None)
test_loader = DataLoader(test_set, batch_size=BATCH_SIZE, shuffle=False)

# create model and load weights from checkpoint
model = Classifier().to(device)
model.load_state_dict(torch.load(model_path))

<All keys matched successfully>

In [ ]:
# Make Prediction
predict = []
model.eval() # set the model to evaluation mode
with torch.no_grad():
    for i, data in enumerate(test_loader):
        inputs = data
        inputs = inputs.to(device)
        outputs = model(inputs)
        _, test_pred = torch.max(outputs, 1) # get the index of the class with the highest probability

        for y in test_pred.cpu().numpy():
            predict.append(y)

In [ ]:
# Save Prediction Results
from datetime import datetime,timezone,timedelta
def save_pred(predict, file):
    ''' Save predictions to specified file '''
    print('Saving results to {}'.format(file))
    
    with open(file, 'w') as f:
      f.write('Id,Class\n')
      for i, y in enumerate(predict):
        f.write('{},{}\n'.format(i, y))


dt1 = datetime.utcnow().replace(tzinfo=timezone.utc)
dt2 = dt1.astimezone(timezone(timedelta(hours=8))) # convert time zone
filename='hero_hw_02_'+dt2.strftime("%Y_%m_%d-%I_%M_%S_%p")+'.csv'
save_pred(predict, filename)         # save prediction file to pred.csv

Saving results to hero_hw_02_2022_09_12-05_52_38_PM.csv


In [ ]:
# Note
# 01. add batch norm
# 02. add dropout

In [ ]:
# Result
# hero_hw_02_2022_09_12-05_52_38_PM.csv
# Score: 0.73622
# Private score: 0.73501

In [ ]:
# Ref
# 01. sample code: https://colab.research.google.com/github/ga642381/ML2021-Spring/blob/main/HW02/HW02-1.ipynb
# 02. kaggle code: https://www.kaggle.com/code/tamakoyl/2021hw02phoneme